# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

import numpy as np
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("Data Wrangling").getOrCreate()

In [3]:
user_log = spark.read.json('data/sparkify_log_small.json')

user_log.createOrReplaceTempView('user_log_table')

# Question 1

Which page did user id ""(empty string) NOT visit?

In [4]:
# TODO: write your code to answer question 1

spark.sql("""
        SELECT DISTINCT page
        FROM user_log_table
        WHERE page
        NOT IN
        (SELECT DISTINCT page
        FROM user_log_table
        WHERE userId = "")
""").show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

You might prefer SQL over data frames because the syntax is clearer especially for people already experienced in SQL.

Spark data frames give us more control. You can break down your queries into smaller steps, which can make debugging easier. You can also cache intermediate results or repartition intermediate results.

# Question 3

How many female users do we have in the data set?

In [5]:
# TODO: write your code to answer question 3

spark.sql("""
    SELECT gender, COUNT(DISTINCT userId) AS count
    FROM user_log_table 
    WHERE gender = 'F'
    GROUP BY gender
""").show()

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
+------+-----+



# Question 4

How many songs were played from the most played artist?

In [6]:
# TODO: write your code to answer question 4

spark.sql("""
    SELECT artist, COUNT(song) AS plays_count
    FROM user_log_table
    WHERE page = 'NextSong'
    GROUP BY artist
    ORDER BY plays_count DESC
    LIMIT 1
""").show()

+--------+-----------+
|  artist|plays_count|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [7]:
# TODO: write your code to answer question 5

is_home = spark.sql("SELECT userID, page, ts, CASE WHEN page = 'Home' THEN 1 ELSE 0 END AS is_home \
            FROM user_log_table WHERE (page = 'NextSong') or (page = 'Home')")

is_home.createOrReplaceTempView("is_home_table")

cumulative_sum = spark.sql("SELECT *, SUM(is_home) OVER \
    (PARTITION BY userID ORDER BY ts DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS period \
    FROM is_home_table")

cumulative_sum.createOrReplaceTempView("period_table")

spark.sql("SELECT AVG(count_results) FROM \
          (SELECT COUNT(*) AS count_results FROM period_table \
GROUP BY userID, period, page HAVING page = 'NextSong') AS counts").show()

+------------------+
|avg(count_results)|
+------------------+
| 6.898347107438017|
+------------------+

